## Install TileDB.CSharp nuget package

In [ ]:
#r "nuget:TileDB.CSharp"

Installed Packages TileDB.CSharp, 2.4.6

### Print out the tiledb core lib version

In [ ]:
System.Console.WriteLine("tiledb core version:{0}", TileDB.ArrayUtil.get_tiledb_version());

tiledb core version:2.4.4


## Create a dense array

### Create dimensions

In [ ]:
var ctx = new TileDB.Context();
var dom = new TileDB.Domain(ctx);
dom.add_int32_dimension("rows",1,4,4);
dom.add_int32_dimension("cols",1,4,4);

### Create the schema

In [ ]:
var schema = new TileDB.ArraySchema(ctx,TileDB.ArrayType.TILEDB_DENSE);
schema.set_domain(dom);
var attr1 = TileDB.Attribute.create_attribute(ctx,"a",TileDB.DataType.TILEDB_INT32);
schema.add_attribute(attr1);

### Delete the array if it exists

In [ ]:
string array_uri = "test_dense_array";
var vfs = new TileDB.VFS(ctx);
if(vfs.is_dir(array_uri))
{
    vfs.remove_dir(array_uri);

}

### Create the array

In [ ]:
TileDB.Array.create(array_uri,schema);

## Write the dense array

### Initialize the data to write

In [ ]:
var data = new TileDB.VectorInt32();
for(int i=1; i<=16; ++i) {
    data.Add(i);
}

### Open the array for write

In [ ]:
var array = new TileDB.Array(ctx,array_uri,TileDB.QueryType.TILEDB_WRITE);
var query = new TileDB.Query(ctx,array,TileDB.QueryType.TILEDB_WRITE);
query.set_layout(TileDB.LayoutType.TILEDB_ROW_MAJOR);
query.set_int32_vector_buffer("a",data);
var status = query.submit();
array.close();
System.Console.WriteLine("write status:{0}",status);

write status:TILEDB_COMPLETED


## Read the array

### Set the subarray

In [ ]:
var subarray = new TileDB.VectorInt32();
subarray.Add(1);
subarray.Add(2); //rows 1,2
subarray.Add(2);
subarray.Add(4);//cols 2,3,4

### Initialize a read buffer, Note that here we use Repeat method

In [ ]:
var read_data = TileDB.VectorInt32.Repeat(0,6); //hold 6 int values

### Open the array and read

In [ ]:
var array_read = new TileDB.Array(ctx,array_uri,TileDB.QueryType.TILEDB_READ);
var query_read = new TileDB.Query(ctx,array_read,TileDB.QueryType.TILEDB_READ);
query_read.set_layout(TileDB.LayoutType.TILEDB_ROW_MAJOR);
query_read.set_int32_subarray(subarray);
query_read.set_int32_vector_buffer("a",read_data);
var status_read = query_read.submit();
array_read.close();
System.Console.WriteLine("read status:{0}", status_read);

read status:TILEDB_COMPLETED


### Print query result

In [ ]:
System.Console.WriteLine("query result:");
System.Console.WriteLine(String.Join(" ",read_data))

query result:
2 3 4 6 7 8


## Clean up

In [ ]:
if(vfs.is_dir(array_uri))
{
    vfs.remove_dir(array_uri);

}